## ***Hypothesis Testing with Insurance Data***

In [4]:
# Import Libraries:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from scipy import stats

In [6]:
# load Data:

df = pd.read_csv('C:\\Users\jnate\\OneDrive\\Documents\\Data Enrichment\\data-enrichment-wk15\\Hypothesis-Testing-\\insurance - insurance.csv')

df.head(5)

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


### Q1. Do smokers have higher insurance charges than non-smokers?

1. State your Null Hypothesis and Alternative Hypothesis:

***A: $H_0$: Smokers pay the same in insurance as non smokers.***

***$H_A$: Smokers pay more in insurance than non smokers.*** 

2.Select the correct test according to the data type and number of samples.

Type of comparison: 2 samples

Data type: Numeric

Hypothesis Test: ***2 Sample T-Test***

3. Test the assumptions of your selected test.

In [14]:
# Separation of smokers an non smokers:

smokers = df.loc[df['smoker']=='yes'].copy()
nonsmokers = df.loc[df['smoker']=='no'].copy()